In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
#path  = 'C:/PycharmProjects/pythonProject1/yjz/Extension for hts/JayCode/Data/'
tr_y = pd.read_pickle("721past_1913.pkl")
ts_y = pd.read_pickle("721future_28.pkl")
y_hist   = tr_y['y']
y_test   = ts_y['y']
fcst = pd.read_pickle("721fcsts_base2cohe.pkl")

In [2]:
(tr_y.iloc[3*1913:4*1913,-1].diff().dropna()**2).mean()

623.428870292887

In [3]:
ts_y.iloc[3*28:,:]

,unique_id,ds,y
84,TOTAL/FOODS/FOODS_1/FOODS_1_004,1914,0
85,TOTAL/FOODS/FOODS_1/FOODS_1_004,1915,15
86,TOTAL/FOODS/FOODS_1/FOODS_1_004,1916,33
87,TOTAL/FOODS/FOODS_1/FOODS_1_004,1917,67
88,TOTAL/FOODS/FOODS_1/FOODS_1_004,1918,118
...,...,...,...
1195203,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1937,0
1195204,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1938,0
1195205,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1939,0
1195206,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1940,0


In [3]:
'''
tags    = pd.read_pickle(f"tags.bin")
l10_12 = ['total/cat_id/dept_id/item_id', 
          'total/state_id/cat_id/dept_id/item_id', 
          'total/state_id/store_id/cat_id/dept_id/item_id']
tags_ = {i:tags[i] for i in l10_12}
ts2 = ts[ts['levels'].isin(['10','11','12'])]
prods = [i.split('/')[-1] for i in tags_[l10_12[0]]]'''

'\ntags    = pd.read_pickle(f"tags.bin")\nl10_12 = [\'total/cat_id/dept_id/item_id\', \n          \'total/state_id/cat_id/dept_id/item_id\', \n          \'total/state_id/store_id/cat_id/dept_id/item_id\']\ntags_ = {i:tags[i] for i in l10_12}\nts2 = ts[ts[\'levels\'].isin([\'10\',\'11\',\'12\'])]\nprods = [i.split(\'/\')[-1] for i in tags_[l10_12[0]]]'

In [31]:
fcst

,lgb_base,lgb_bu,lgb_td,lgb_mint,ets_base,ets_bu,ets_td,ets_mint
0,5.689794,6.307602,5.689794,5.288739,6.747322,6.392221,6.747322,6.949374
1,4.679130,5.639605,4.679130,4.605017,6.525307,6.303542,6.525307,6.231106
2,4.798928,5.449359,4.798928,4.613737,6.398088,6.313164,6.398088,6.544447
3,5.980217,6.054023,5.980217,5.405183,6.448675,6.259084,6.448675,6.743307
4,5.048564,5.671963,5.048564,5.003933,5.990921,6.079106,5.990921,6.804783
...,...,...,...,...,...,...,...,...
1195203,0.177103,0.177103,0.108058,0.088706,0.076043,0.076043,0.083374,0.031334
1195204,0.186536,0.186536,0.105750,0.093228,0.074525,0.074525,0.087415,0.068989
1195205,0.174125,0.174125,0.095396,0.065864,0.074324,0.074324,0.079272,0.095974
1195206,0.242970,0.242970,0.180732,0.142686,0.073504,0.073504,0.081925,0.121862


## Functions for RMSSE

In [4]:
import numpy as np
from sklearn.metrics import mean_squared_error as mse

def _get_df(df:pd.DataFrame, n:int = 42840, gap:int=28):
    df_csv = []
    for i in tqdm(range(n)):
        df_csv.append(df[i*gap:(i+1)*gap].reset_index(drop=True))
    df_csv = pd.concat(df_csv,axis=1,ignore_index= True)
    return df_csv
def rmsse(yts, fcst, ytr, n:int = 42840):
    df_l = []
    scaler =  (ytr.diff().dropna()**2).mean(axis = 0)
    for i in tqdm(range(n)):
        mse_ = mse(yts.iloc[:,i],fcst.iloc[:,i])
        df_l.append(np.sqrt(mse_/scaler[i]))
    return df_l   
y_tr = _get_df(y_hist,gap = 1913,n = 42686)
y_ts = _get_df(y_test,n = 42686)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:03<00:00, 11416.98it/s]


## RMSSE:

In [5]:
y_ts

,0,1,2,3,4,5,6,7,8,9,...,42676,42677,42678,42679,42680,42681,42682,42683,42684,42685
0,4,9,7,0,14,12,1,1,1,5,...,0,0,1,1,0,2,0,0,0,0
1,5,4,13,15,9,15,7,11,5,12,...,0,0,0,0,1,0,0,0,0,0
2,7,3,9,33,27,10,1,10,8,7,...,0,0,1,1,0,0,0,0,0,0
3,1,4,1,67,14,13,1,3,9,9,...,2,0,1,0,0,0,0,0,0,0
4,9,5,2,118,14,8,10,13,5,10,...,0,0,1,1,0,2,1,0,0,1
5,3,7,4,101,33,12,2,13,4,14,...,0,0,1,2,4,1,0,0,0,0
6,9,2,7,132,19,6,1,16,4,8,...,3,1,1,1,0,1,0,0,0,0
7,1,5,9,119,17,7,1,6,5,5,...,0,0,1,0,0,0,0,1,0,0
8,10,6,2,156,11,26,2,13,5,16,...,0,0,0,0,0,0,1,0,0,0
9,3,3,3,116,16,10,3,10,6,12,...,0,0,0,2,1,0,1,0,0,0


In [6]:
y_tr

,0,1,2,3,4,5,6,7,8,9,...,42676,42677,42678,42679,42680,42681,42682,42683,42684,42685
0,6,4,14,0,34,16,0,0,0,7,...,0,0,1,0,0,0,0,0,0,0
1,6,5,8,0,32,9,0,0,0,1,...,0,0,1,0,5,2,0,1,0,1
2,4,7,3,0,13,20,0,0,0,5,...,0,0,1,0,1,0,0,1,0,0
3,6,4,6,0,20,16,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
4,7,3,3,0,10,14,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,3,5,4,0,27,10,1,0,4,18,...,0,0,0,0,0,1,0,0,0,0
1909,6,5,4,0,12,4,1,0,4,10,...,0,0,1,0,0,0,0,0,1,0
1910,2,3,3,0,15,14,1,0,6,9,...,1,0,1,1,0,0,0,1,1,0
1911,16,3,11,0,38,10,1,0,5,8,...,1,0,0,2,0,0,0,0,1,0


In [7]:
i = 'lgb_base'
df = _get_df(fcst[i], n = 42686)
df

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:03<00:00, 11599.67it/s]


,0,1,2,3,4,5,6,7,8,9,...,42676,42677,42678,42679,42680,42681,42682,42683,42684,42685
0,5.689794,2.188528,4.439871,0.208418,15.118921,8.123409,1.692846,0.441812,6.177771,9.473888,...,0.358518,0.125571,0.487637,0.471610,0.125571,0.301600,0.148863,0.210391,0.385527,0.125571
1,4.679130,4.150071,4.689257,0.256420,15.851895,8.850806,1.287959,0.586008,3.357940,9.412122,...,0.238299,0.127944,0.525776,0.553280,0.127944,0.623322,0.127944,0.168800,0.269261,0.127944
2,4.798928,4.114768,6.618687,4.856871,13.608904,10.128646,2.839844,3.273207,3.865219,9.518998,...,0.216084,0.125870,0.392239,0.438619,0.328142,0.414948,0.125870,0.155488,0.251164,0.125870
3,5.980217,3.530227,7.388255,20.119639,17.007754,9.364186,2.182627,4.741831,5.579967,8.272455,...,0.206394,0.130216,0.571126,0.578931,0.250414,0.323201,0.130216,0.168923,0.253141,0.130216
4,5.048564,3.702424,6.919829,45.697817,18.773698,11.839938,2.159676,4.091495,7.029738,8.298148,...,0.773183,0.139859,0.811582,0.631523,0.241978,0.344225,0.139859,0.205016,0.260894,0.139859
5,8.247916,4.817758,7.301357,88.000520,20.642775,12.078862,5.205704,7.874821,7.106865,10.287515,...,0.649470,0.169652,1.018123,0.934418,0.318724,0.972735,0.431163,0.169652,0.271485,0.431163
6,6.159383,6.409254,6.678464,87.578116,25.171901,13.165732,4.750904,10.014469,7.028007,11.496851,...,0.519997,0.199208,1.350162,1.286237,1.425117,1.099785,0.429461,0.199208,0.199208,0.429461
7,6.263591,5.082343,6.676917,86.269300,18.143720,9.973163,3.248808,10.286752,4.918472,8.859958,...,1.081850,0.412169,0.917353,0.938223,0.809332,0.942203,0.268262,0.189578,0.189578,0.268262
8,4.828214,3.920717,7.523390,84.975060,19.458289,9.891918,2.873845,9.341923,5.517999,8.881679,...,0.703655,0.348882,0.884062,0.739491,0.712123,0.639024,0.276034,0.420098,0.200333,0.276034
9,6.094202,4.337865,4.789551,105.327995,15.613408,13.627562,2.000493,9.435104,5.178199,10.262249,...,0.575622,0.211804,0.489401,0.432420,0.503584,0.353638,0.375963,0.255531,0.146268,0.188170


In [8]:
i = 'lgb_base'
df = _get_df(fcst[i], n = 42686)
pd.DataFrame({f'{i}_rmsse':rmsse(yts = y_ts,fcst=df,ytr = y_tr, n = 42686)})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:26<00:00, 1605.71it/s]


,lgb_base_rmsse
0,0.496969
1,0.835554
2,0.790961
3,1.150779
4,0.833183
...,...
42681,1.079308
42682,0.540944
42683,0.621724
42684,2.326800


In [9]:
%%time
fcst = pd.read_pickle("721fcsts_base2cohe.pkl")
rmsse_l = []
for i in fcst.columns:
    df = _get_df(fcst[i], n = 42686)
    df = pd.DataFrame({f'{i}_rmsse':rmsse(yts = y_ts,fcst=df,ytr = y_tr, n = 42686)})
    rmsse_l.append(df)
    print(f"{i} Finished！")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:26<00:00, 1598.95it/s]


lgb_base Finished！


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:27<00:00, 1569.72it/s]


lgb_bu Finished！


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:26<00:00, 1627.85it/s]


...

ets_bu Finished！


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:26<00:00, 1633.04it/s]


ets_td Finished！


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [00:26<00:00, 1614.70it/s]

ets_mint Finished！
Wall time: 4min 27s


In [10]:
all_r = pd.concat(rmsse_l,axis=1)
all_r.to_pickle("721h1_RMSSEs_tdfp.pkl")
all_r

,lgb_base_rmsse,lgb_bu_rmsse,lgb_td_rmsse,lgb_mint_rmsse,ets_base_rmsse,ets_bu_rmsse,ets_td_rmsse,ets_mint_rmsse
0,0.496969,0.491215,0.496969,0.483473,0.494260,0.481727,0.494260,0.494980
1,0.835554,0.811777,0.835554,0.802532,0.751217,0.730490,0.751217,0.749245
2,0.790961,0.788101,0.790961,0.764541,0.693869,0.690631,0.693869,0.707982
3,1.150779,1.225750,1.150779,1.157231,0.939762,1.138942,0.939762,0.964001
4,0.833183,0.823551,0.833183,0.815368,0.879527,0.841358,0.879527,0.809016
...,...,...,...,...,...,...,...,...
42681,1.079308,1.079308,1.053242,1.072878,0.989247,0.989247,0.977628,0.994391
42682,0.540944,0.540944,0.520966,0.526615,0.500271,0.500271,0.498052,0.510732
42683,0.621724,0.621724,0.565899,0.575280,0.538230,0.538230,0.545314,0.545500
42684,2.326800,2.326800,2.286208,2.293298,2.228416,2.228416,2.243456,2.241407


In [12]:
all_r = pd.concat(rmsse_l,axis=1)
all_r.to_pickle("721h1_RMSSEs_tdfp.pkl")
all_r

,lgb_base_rmsse,lgb_bu_rmsse,lgb_td_rmsse,lgb_mint_rmsse,ets_base_rmsse,ets_bu_rmsse,ets_td_rmsse,ets_mint_rmsse
0,0.496969,0.491215,0.496969,0.483473,0.494260,0.481727,0.494260,0.494980
1,0.835554,0.811777,0.963397,0.802532,0.751217,0.730490,0.848817,0.749245
2,0.790961,0.788101,1.224018,0.764541,0.693869,0.690631,1.256026,0.707982
3,1.150779,1.225750,4.227228,1.157231,0.939762,1.138942,4.229478,0.964001
4,0.833183,0.823551,2.174971,0.815368,0.879527,0.841358,2.164957,0.809016
...,...,...,...,...,...,...,...,...
42681,1.079308,1.079308,1.066718,1.072878,0.989247,0.989247,1.013164,0.994391
42682,0.540944,0.540944,0.532703,0.526615,0.500271,0.500271,0.500175,0.510732
42683,0.621724,0.621724,0.605882,0.575280,0.538230,0.538230,0.524587,0.545500
42684,2.326800,2.326800,2.213522,2.293298,2.228416,2.228416,2.296091,2.241407


In [18]:
np.where(all_r["lgb_td_rmsse"] == all_r["lgb_td_rmsse"].max())

(array([16772], dtype=int64),)

In [12]:
all_r.mean()#.iloc[16772,]

lgb_base_rmsse    0.935863
lgb_bu_rmsse      0.943704
lgb_td_rmsse      0.923867
lgb_mint_rmsse    0.923328
ets_base_rmsse    0.904904
ets_bu_rmsse      0.904141
ets_td_rmsse      0.906681
ets_mint_rmsse    0.897178
dtype: float64

In [19]:
all_r.iloc[16772,]

lgb_base_rmsse      0.606158
lgb_bu_rmsse        0.606158
lgb_td_rmsse      999.315243
lgb_mint_rmsse      0.507829
ets_base_rmsse      0.500671
ets_bu_rmsse        0.500671
ets_td_rmsse      986.197319
ets_mint_rmsse      0.534760
Name: 16772, dtype: float64

In [21]:
fcst.iloc[16772]

lgb_base    23.049472
lgb_bu      19.643465
lgb_td       2.544231
lgb_mint    21.742515
ets_base    23.454813
ets_bu      22.081604
ets_td       3.019504
ets_mint    23.905410
Name: 16772, dtype: float64

In [24]:
np.mean(all_r.iloc[:3049,:])

c:\py\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


lgb_base_rmsse    0.918876
lgb_bu_rmsse      0.970923
lgb_td_rmsse      2.256668
lgb_mint_rmsse    0.910020
ets_base_rmsse    0.949191
ets_bu_rmsse      0.945819
ets_td_rmsse      2.256260
ets_mint_rmsse    0.911686
dtype: float64